In [17]:
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
import warnings
from sqlalchemy import create_engine, text
import os
import pickle
from datetime import datetime
import json


In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
warnings.filterwarnings("ignore")

In [5]:
TABLE_NAME = "transactions_data"
DB_NAME = "client_data"

In [6]:
DB_USER = os.environ.get("DB_USER")
DB_PW = os.environ.get("DB_PW")
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")
print(DB_USER)

root


In [7]:
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}")

In [8]:
data_query = f"SELECT * from {DB_NAME}.{TABLE_NAME}"
raw_data = pd.read_sql(data_query, engine)

In [9]:
engine.dispose()

In [10]:
raw_data["merchant_country_code"] = raw_data["merchant_country_code"].astype(str).str[:3]

In [11]:
mean_by_cardid = raw_data.groupby('card_id')['settlement_amount'].mean()
mean_by_cardid = mean_by_cardid.reset_index() 
mean_by_cardid.rename(columns = {"settlement_amount": "mean_by_cardid"}, inplace = True)

raw_data = raw_data.merge(mean_by_cardid, on="card_id", how='left')

In [12]:
raw_data = raw_data[~raw_data["settlement_amount"].isna()]

In [13]:
le = preprocessing.LabelEncoder()
raw_data[["transaction_type_enc", "settlement_currency_enc", "merchant_country_code_enc", "transaction_code_enc", "transaction_currency_enc", "payment_provider_enc", "action_code_enc", "cycle_enc"]] = raw_data[["transaction_type", "settlement_currency", "merchant_country_code", "transaction_code", "transaction_currency", "payment_provider", "action_code", "cycle"]].astype(str).apply(le.fit_transform)

In [19]:
mappings = {}

In [24]:
for enc_col in ["transaction_type", "settlement_currency", "merchant_country_code", "transaction_code", "transaction_currency", "payment_provider", "action_code", "cycle"]:
    mappings_df = raw_data[[enc_col, f"{enc_col}_enc"]].drop_duplicates()
    mappings[enc_col] = {item[0]: item[1] for item in json.loads(mappings_df.to_json(orient="split"))["data"]} 

In [25]:
with open("mappings.txt", "w") as file:
    file.write(json.dumps(mappings))

{'transaction_type': {'Purchase Domestic': 0,
  'Withdrawal Domestic': 5,
  'Refund Domestic': 2,
  'Purchase International': 1,
  'Unique Transfer Domestic': 4,
  'Refund International': 3,
  'Withdrawal International': 6},
 'settlement_currency': {826: 0},
 'merchant_country_code': {'826': 15,
  '250': 2,
  '372': 5,
  '831': 16,
  '840': 17,
  '528': 7,
  '276': 3,
  '196': 0,
  '344': 4,
  '724': 11,
  '208': 1,
  '554': 8,
  '620': 10,
  '756': 12,
  '764': 13,
  '440': 6,
  '566': 9,
  '792': 14},
 'transaction_code': {'1': 3,
  '01': 0,
  '4': 5,
  '20': 4,
  '04': 1,
  '05': 2,
  '5': 6},
 'transaction_currency': {826: 4,
  784: 3,
  124: 0,
  978: 6,
  764: 2,
  840: 5,
  566: 1},
 'payment_provider': {'': 0},
 'action_code': {0.0: 0, None: 1},
 'cycle': {4.0: 3, 2.0: 1, 3.0: 2, 1.0: 0, 5.0: 4, 6.0: 5, None: 6}}

In [ ]:
le.classes_

In [ ]:
raw_data["transaction_date"] = pd.to_datetime(raw_data["transaction_date"])

raw_data["day"] = raw_data["transaction_date"].dt.day
raw_data["month"] = raw_data["transaction_date"].dt.month
raw_data["year"] = raw_data["transaction_date"].dt.year

In [ ]:
pred_cols = ["transaction_amount", "merchant_country_code", "settlement_amount", "transaction_code", "transaction_currency", "settlement_currency", "transaction_type", "payment_provider", "action_code", "cycle", "mean_by_cardid", "day", "month", "year"]
raw_data = raw_data[pred_cols]

In [ ]:
model = IsolationForest(contamination=0.01, random_state=42)
model.fit(raw_data)

In [ ]:
def get_latest_version_model():
    models = os.listdir("models")
    if not models:
        return 0
    models = list(filter(lambda x: x.endswith(".pkl"), models))
    latest_model = sorted(models, reverse=True)[0]
    latest_version = latest_model.split("_")[1]
    return latest_version

def version_manager(version, change):
    if version == 0:
        return "1.0.0"
    
    if change.lower() == "major":
        version = ".".join([str(int(version.split(".")[0]) + 1), version.split(".")[1],version.split(".")[2]])
    elif change.lower() == "minor":
        version = ".".join([version.split(".")[0], str(int(version.split(".")[1]) + 1), version.split(".")[2]])
    elif change.lower() == "iteration":
        version = ".".join([version.split(".")[0], version.split(".")[1], str(int(version.split(".")[2]) + 1)])
    else:
        raise ValueError("Invalid change type")
    return version

In [ ]:
current_version = version_manager(get_latest_version_model(), "iteration")
today = datetime.now().strftime("%Y-%m-%d")
today = today.replace("-",  "")

In [ ]:
model_path = f"models/IFMODEL_{current_version}.pkl"

os.makedirs("models", exist_ok=True)
pickle.dump(model, open(model_path, "wb"))
